# SD Simulations with Submission System


## Imports

In [1]:
import os
#for analysis
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from pygromos.files.gromos_system import Gromos_System
from pygromos.files.blocks import imd_blocks
from pygromos.data.simulation_parameters_templates import template_emin

## Input files

In [2]:
root_dir = project_dir = "../example_files/SD_Simulation/"
root_in_dir = root_dir+"/SD_input"
cnf_path = root_in_dir+"/6J29_unitedatom_optimised_geometry.cnf"
top_path = root_in_dir + "/6J29.top"
sys_name = "6J29"

#External imd_changes:
grom_system = Gromos_System(in_cnf_path=cnf_path, in_top_path=top_path, in_imd_path=template_emin,
                            system_name=sys_name, work_folder=root_in_dir)

grom_system.cnf.add_empty_box()


## RUN Emin

In [3]:
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL as subSystem
from pygromos.simulations.modules.preset_simulation_modules import emin
step_name  = "a_vacuum_emin"

#Some special adaptation
grom_system.imd.BOUNDCOND.NTB =0
grom_system.imd.BOUNDCOND.NDFMIN = 0
grom_system.imd.WRITETRAJ = imd_blocks.WRITETRAJ()
grom_system.imd.WRITETRAJ.NTWX = 100
grom_system.imd.WRITETRAJ.NTWE = 100
grom_system.prepare_for_simulation()

In [4]:

grom_system.save(root_dir+"/test.obj", safe=False)

'../example_files/SD_Simulation//test.obj'

In [5]:
grom_system = Gromos_System.load(root_dir+"/test.obj")

In [6]:
grom_system.solute_info._asdict()

{'names': ['6J29'], 'number': 1, 'positions': [1], 'number_of_atoms': 27}

In [7]:
grom_system.cnf.visualize()

AttributeError: 'Cnf' object has no attribute 'visualize'

In [8]:
from pygromos.simulations.modules.preset_simulation_modules import emin


emin_gromos_system = emin(in_gromos_system=grom_system, 
                                 step_name=step_name, 
                                 submission_system=subSystem(), 
                                 in_imd_path=None)
jobID = emin_gromos_system._last_jobID
emin_gromos_system


################################################################################

a_vacuum_emin
################################################################################



Traceback (most recent call last):
  File "/home/mlehner/PyGromosTools/pygromos/simulations/modules/general_simulation_modules.py", line 130, in simulation
    gromos_system.rebase_files()
  File "/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py", line 880, in rebase_files
    self.write_files()
  File "/home/mlehner/PyGromosTools/pygromos/files/gromos_system/gromos_system.py", line 1120, in write_files
    file_obj.write(file_obj.path)
  File "/home/mlehner/PyGromosTools/pygromos/files/_basics/_general_gromos_file.py", line 306, in write
    self._write_to_file(out_path=out_path, content_str=str(self))
  File "/home/mlehner/PyGromosTools/pygromos/files/_basics/_general_gromos_file.py", line 324, in _write_to_file
    raise IOError("Could not find directory to write to: " + str(os.path.dirname(out_path)))
OSError: Could not find directory to write to: ../example_files/SD_Simulation//SD_input/a_vacuum_emin/input

During handling of the above exception, another e

Exception: Could not prepare the command block
	Could not prepare the gromos System
	Could not find directory to write to: ../example_files/SD_Simulation//SD_input/a_vacuum_emin/input

In [9]:
emin_gromos_system.cnf.visualize()

NameError: name 'emin_gromos_system' is not defined

In [10]:
emin_gromos_system

NameError: name 'emin_gromos_system' is not defined

In [11]:
emin_gromos_system.work_folder+"/analysis/data"

NameError: name 'emin_gromos_system' is not defined

## RUN SD Simulation

In [12]:
from pygromos.simulations.modules.preset_simulation_modules import sd

step_name  = "b_vacuum_sd"

sd_gromos_system  = sd(in_gromos_system=emin_gromos_system,
                              step_name=step_name, 
                       submission_system=subSystem(nmpi=1),
                      equilibration_runs=1, simulation_runs=3)
jobID = sd_gromos_system._last_jobID

sd_gromos_system

sd_gromos_system.cnf.visualize()

NameError: name 'emin_gromos_system' is not defined

## Further Analysis:
Transform the trc into a pdb traj to check it out with pymol :!)

In [13]:
#final analysis dir:
from pygromos.utils import bash

out_ana = project_dir+"/c_ana"
if(not os.path.exists(out_ana)):
    bash.make_folder(out_ana)

mkdir: cannot create directory ‘../example_files/SD_Simulation//c_ana’: No such file or directory


OSError: could not make folder:
 ../example_files/SD_Simulation//c_ana

### Coordinate Analysis

In [14]:
from pygromos.files.trajectory.trc import Trc

trc = sd_gromos_system.trc
trc.write_pdb(out_ana+"/sd_traj.pdb", cnf_file=cnf_path)#grom_system.cnf.path)
trc

NameError: name 'sd_gromos_system' is not defined

In [15]:
traj = trc.visualize(grom_system.cnf)
traj

NameError: name 'trc' is not defined

### Energy Analysis

In [16]:
from pygromos.files.trajectory.tre import Tre

tre = sd_gromos_system.tre
tre


NameError: name 'sd_gromos_system' is not defined

In [17]:
#Plot Potential Energies
V_tot = np.array(list(map(lambda x: x[2], tre.database.totals)))
step = len(tre.database.TIMESTEP_time)//10

plt.plot(tre.database.TIMESTEP_time, V_tot)
plt.xticks(np.round(list(tre.database.TIMESTEP_time[::step]),2))
plt.xlabel("$t~[ps]$")
plt.ylabel("$V~[kJ]$")
plt.title("V total timeseries")
plt.savefig(out_ana+"/potential_energy_timeseries.png")

NameError: name 'tre' is not defined